In [10]:
import Embed, EmbedHelper, qiskit
from qiskit import QuantumCircuit, QuantumProgram
import heapq as H

In [8]:
Q_program = QuantumProgram()
testcoupling5 = {0: (1,2,3,4), 1:(5,9), 2:(6,10), 3:(7,11), 4:(8,12)}
q5 = Q_program.create_quantum_register("q5", 15)
c5 = Q_program.create_classical_register("c5", 15)
embedtest5 = Q_program.create_circuit("qcircuit5", [q5], [c5])

embedtest5.cx(q5[0],q5[1])
embedtest5.cx(q5[0],q5[2])
embedtest5.h(q5[0])
embedtest5.cx(q5[0],q5[3])
embedtest5.cx(q5[0],q5[5])
embedtest5.cx(q5[5],q5[6])
embedtest5.cx(q5[5],q5[7])
embedtest5.cx(q5[5],q5[8])
embedtest5.cx(q5[5],q5[10])
embedtest5.cx(q5[10],q5[11])
embedtest5.cx(q5[10],q5[12])
embedtest5.cx(q5[10],q5[13])

In [39]:
mapA = {0:0,1:1,2:2,3:3,5:4,6:8,7:12,8:10,10:9,11:7,12:11,13:5}
mapB = {0:4,1:1,2:10,3:9,5:0,6:8,7:12,8:2,10:3,11:7,12:11,13:5}
Sw = [(4,0),(10,2),(9,1),(1,0),(0,3),(1,0),(9,1)]

helpers = EmbedHelper.EmbedHelper(embedtest5, testcoupling5)
UC = helpers.UndirectedCoupling

In [231]:
def score(coupling, targets, node):
    score = 0
    #print(coupling, targets, node)
    for i in UC[node]:
        if i in targets:
            
            score = score + 1
    return score

def MST(coupling, targs):
    MST = {}
    
    if len(targs) == 0:
        assert Exception
    
    accessible = []
    visited = set()
    targets = set(targs)
    target = targets.pop()
    while len(targets) > 0:
        
        visited.add(target)
        tscore = score(UC, targs, target)
        
        for i in UC[target]:
            if i not in visited:
                Score = float(1/(score(UC, targs, i) + tscore))
                H.heappush(accessible, (Score, (i, target)))
     
        Min =  H.heappop(accessible)
        VisitedNode = Min[1][1]
        NewNode = Min[1][0]
        
        if NewNode in MST:
            assert Exception
        if VisitedNode not in MST:
            MST[VisitedNode] = (NewNode,)
        else:      
            MST[VisitedNode] = MST[VisitedNode] + (NewNode,)    
        MST[NewNode] = (VisitedNode,)
      
        target = Min[1][0]
        if target in targets:
            targets.remove(target)
      
    return MST

def hasCycles():
    available = set()
    visited = set()
    node = list(UC)[0]
    traceback = {}
    while len(available) > 0:
        visited.add(node)
        for i in UC[node]:
            if (i in visited) and (traceback[node] != i) :
                return True
            available.add((node, i))
        newnode = available.pop()  
        traceback[newnode[1]] = newnode[0]
        node = newnode[1]

    return False

    

In [232]:
mst = MST(testcoupling5, mapA.values())
print(mst)
print(hasCycles())

{0: (1, 2, 3, 4), 1: (5, 9, 0), 2: (6, 10, 0), 3: (7, 11, 0), 4: (8, 12, 0), 5: (1,), 6: (2,), 7: (3,), 8: (4,), 9: (1,), 10: (2,), 11: (3,), 12: (4,)}
12
{0: (1, 2, 3, 4), 1: (5, 9, 0), 2: (6, 10, 0), 3: (7, 11, 0), 4: (8, 12, 0), 5: (1,), 6: (2,), 7: (3,), 8: (4,), 9: (1,), 10: (2,), 11: (3,), 12: (4,)}
{0: (1, 3, 4, 2), 1: (0, 5, 9), 2: (0, 6, 10), 3: (0, 7, 11), 4: (0, 8, 12), 5: (1,), 6: (2,), 7: (3,), 8: (4,), 9: (1,), 10: (2,), 11: (3,), 12: (4,)}
False
